In [23]:
! export PYTHONPATH=../

In [24]:
from transformers import BartTokenizerFast

from termcolor import colored

import random

import json
import os

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [16]:
import difflib


In [122]:
# path_to_results = '/projects/bdata/tmp/autumn-salad-68/eval-preds-160000.jsonl'
# path_to_results = '/projects/bdata/tmp/eval-preds-overnight-sad.jsonl'
path_to_results = '/projects/bdata/tmp/eval-preds-nopt-20201012.jsonl'

In [123]:
data = []
with open(path_to_results, 'r') as f:
    for l in f:
        data.append(json.loads(l))

In [124]:
len(data)

3495

In [125]:

data[0].keys()

dict_keys(['input', 'label', 'prediction', 'input_ids', 'label_ids', 'pred'])

In [130]:
def display_diff(string1, string2):
    for text in difflib.unified_diff(string1.split("\n"), string2.split("\n")):
        if text[:3] not in ('+++', '---', '@@ '):
            if text.startswith('-'):
                print(colored(text, 'red'))
            elif text.startswith('+'):
                print(colored(text, 'green'))
            else:
                print(text)

In [131]:
example = random.choice(data)

In [132]:
display_diff(example["input"], example["label"])

-data_numerical = data[numerical_columns]
+test_numerical=test[numerical_columns]


In [133]:
display_diff(example["input"], example["prediction"])

-data_numerical = data[numerical_columns]
+_numerical = data[data[data_columns]]


In [134]:
display_diff(example["prediction"], example["label"])

-_numerical = data[data[data_columns]]
+test_numerical=test[numerical_columns]


# Tokenize codes and insert space in between

In [67]:
from tokenize import tokenize, untokenize, NUMBER, STRING, NAME, OP
from io import BytesIO

import re

In [120]:
def insert_space_to_tokenized_code(string):
    try:
        g = tokenize(BytesIO(string.encode('utf-8')).readline)

        result = []

        for toknum, tokval, _, _, _ in g:
            result.append(tokval)
        result = result[1:]
    except:
        result = string.split()
    return ' '.join(result)

In [121]:
print(insert_space_to_tokenized_code(example["input"]))

num_classes = 120 
 resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'  


# Rouge

In [62]:
from rouge import Rouge

In [63]:
rouge = Rouge()

In [135]:
predictions = [d["prediction"] for d in data]

In [139]:
labels = [d["label"] for d in data]

In [140]:
len(labels)

3495

In [141]:
len(predictions)

3495

In [144]:
rouge.get_scores(labels, predictions, avg=True, ignore_empty=True)

{'rouge-1': {'f': 0.4190720264974113,
  'p': 0.43828403251010034,
  'r': 0.4271335497886075},
 'rouge-2': {'f': 0.22649796913324916,
  'p': 0.2381446413098304,
  'r': 0.2291421860837875},
 'rouge-l': {'f': 0.4270384373171943,
  'p': 0.42568682527942153,
  'r': 0.45093692885546993}}

# Rouge with special tokenization

In [145]:
predictions  = [insert_space_to_tokenized_code(d["prediction"]) for d in data]

In [146]:
labels = [insert_space_to_tokenized_code(d["label"]) for d in data]

In [147]:
rouge.get_scores(labels, predictions, avg=True, ignore_empty=True)

{'rouge-1': {'f': 0.4687167032767533,
  'p': 0.468177090278831,
  'r': 0.5191222396400891},
 'rouge-2': {'f': 0.29837637409903556,
  'p': 0.30335970640620324,
  'r': 0.3172249710736497},
 'rouge-l': {'f': 0.4853926578365272,
  'p': 0.4628293249597014,
  'r': 0.5448147807474536}}